# Deep Learning - Lab Exercise 2

### Student 1: Ying LAI. Student 2: Yingjie LIU

**WARNING:** you must have finished the previous exercise before this one as you will re-use parts of the code.

In the first lab exercise, we built a simple linear classifier.
Although it can give reasonable results on the MNIST dataset (~92.5% of accuracy), deeper neural networks can achieve more the 99% accuracy.
However, it can quickly become really impracical to explicitly code forward and backward passes.
Hence, it is useful to rely on an auto-diff library where we specify the forward pass once, and the backward pass is automatically deduced from the computational graph structure.

In this lab exercise, we will build a small and simple auto-diff lib that mimics the autograd mechanism from Pytorch (of course, we will simplify a lot!)


In [2]:
# import libs that we will use
import os
import numpy as np
import matplotlib.pyplot as plt
import math

# To load the data we will use the script of Gaetan Marceau Caron
# You can download it from the course webiste and move it to the same directory that contains this ipynb file
import dataset_loader

%matplotlib inline

In [3]:
# Download mnist dataset 
if("mnist.pkl.gz" not in os.listdir(".")):
    # this link doesn't work any more,
    # seach on google for the file "mnist.pkl.gz"
    # and download it
    !wget http://deeplearning.net/data/mnist/mnist.pkl.gz

# if you have it somewhere else, you can comment the lines above
# and overwrite the path below
mnist_path = "./mnist.pkl.gz"

In [4]:
# load the 3 splits
train_data, dev_data, test_data = dataset_loader.load_mnist(mnist_path)

## Computation Graph

Instead of directly manipulating numpy arrays, we will manipulate abstraction that contains:
- a value (i.e. a numpy array)
- a bool indicating if we wish to compute the gradient with respect to the value
- the gradient with respect to the value
- the operation to call during backpropagation

There will be two kind of nodes:
- ComputationGraphNode: a generic computation node
- Parameter: a computation node that is used to store parameters of the network. Parameters are always leaf nodes, i.e. they cannot be build from other computation nodes.

Our implementation of the backward pass will be really simple and incorrect in the general case (i.e. won't work with computation graph with loops).
We will just apply the derivative function for a given tensor and then call the ones of its antecedents, recursively.
This simple algorithm is good enough for this exercise.

Note that a real implementation of backprop will store temporary values during forward that can be used during backward to improve computation speed. We do not do that here.


In [5]:
class ComputationGraphNode(object):
    
    def __init__(self, data, require_grad=False):
        # we initialise the value of the node and the grad
        if(not isinstance(data, np.ndarray)):
            data = np.array(data)
        self.value = data
        self.grad = None
        
        self.require_grad = require_grad
        self.func = None
        self.input_nodes = None
        self.func_parameters = []
    
    def set_input_nodes(self, *nodes):
        self.input_nodes = list(nodes)

    def set_func_parameters(self, *func_parameters):
        self.func_parameters = list(func_parameters)
    
    def set_func(self, func):
        self.func = func

    def zero_grad(self):
        if self.grad is not None:
            self.grad.fill(0)

    def set_gradient(self, gradient):
        """
        Accumulate gradient for this tensor
        """
        if gradient.shape != self.value.shape:
            print(gradient.shape, self.value.shape)
            raise RuntimeError("Invalid gradient dimension")
        if self.grad is None:
            self.grad = gradient
        else:
            self.grad += gradient
    
    def backward(self, g=None):
        if g is None:
            g = self.value.copy()
            g.fill(1.)
        self.set_gradient(g)
        if self.func is not None:
            grad_list = self.func.backward(*(self.input_nodes + self.func_parameters + [g]))
            for input_node, ngrad in zip(self.input_nodes, grad_list):
                input_node.backward(ngrad)
    
    def __add__(self, y):
        if not isinstance(y, ComputationGraphNode):
            y = ComputationGraphNode(y)
        return Addition()(self, y)

    def __getitem__(self, slice):
        return Selection()(self, slice)

    def __str__(self):
        return self.value.__str__()

    def __repr__(self):
        return self.value.__str__()

class Parameter(ComputationGraphNode):
    def __init__(self, data, name="default"):
        super().__init__(data, require_grad=True)
        self.name  = name

    def backward(self, g=None):
        if g is not None:
            self.set_gradient(g)

The class `Operation` is a class that three methods you should reimplement only the forward and the backward methods.
* The `forward` method compute the function w.r.t inputs and return a new node that must contains information for backward pass.
* The `backward` functions compute the gradient of the function w.r.t gradient of the output and other informations (forward pass input, parameter of the function...).**It should return a tuple**

For better understanding below two operation are implemented, the selection and the addition (notice that it should not works yet since we do not defined what is a node)

In [6]:
class Operation(object):
    @staticmethod
    def forward(*args):
        raise NotImplementedError("It is an abstract method")
    
    def __call__(self, *args):
        output_node = self.forward(*args)
        output_node.set_func(self)
        return output_node
        
    @staticmethod
    def backward(*args):
        pass
class Addition(Operation):
    @staticmethod
    def forward(x, y):
        output_array = x.value + y.value
        output_node = ComputationGraphNode(output_array)
        output_node.set_input_nodes(x, y)
        return output_node

    @staticmethod
    def backward(x, y, gradient):
        return (gradient, gradient)

class Selection(Operation):
    @staticmethod
    def forward(x, slice):
        np_x = x.value

        output_array = np_x.__getitem__(slice)
        
        output_node = ComputationGraphNode(output_array)
        output_node.set_input_nodes(x)
        output_node.set_func_parameters(slice)

        return output_node
        
    @staticmethod
    def backward(x, slice, gradient):
        np_x = x.value

        cgrad = np_x.copy()
        cgrad.fill(0)
        cgrad.__setitem__(slice, gradient)
        
        return cgrad,

**Question 1** Complete the following class 

In [7]:
class ReLU(Operation):
    @staticmethod
    def forward(x):
        # we copy the value of the input node
        np_x = x.value.copy()

        # set negative elements to zero
        np_x[np_x < 0] = 0 # notice we consider strictly < 0 

        # we create the output node needing only the node x
        output_node = ComputationGraphNode(np_x)
        output_node.set_input_nodes(x)
        
        return output_node
        
    @staticmethod
    def backward(x, gradient):
        
        # we copy the value of the input node
        np_x = x.value.copy()

        # np_x[np_x <= 0] = 0 # consider <= 0
        # np_x[np_x > 0] = 1
        # grad = gradient * np_x
        # we multiply the gradient by the gradient of the ReLU
        grad = (np_x > 0).astype(np_x.dtype) * gradient      

        return (grad,)
        raise NotImplementedError

We recall that :  $$tanh(x)= \frac{e^{z} - e^{-z}}{e^{z} + e^{-z}}$$ 

However we can have stability issues if $||z||$ is large, e.g. $e^{10000}$ will lead to computation error or infinity. Indeed in python using numpy:


>np.exp(10000)


will leads to :

>/tmp/ipykernel_7784/2473798304.py:1: RuntimeWarning: overflow encountered in exp
>np.exp(10000)
>
>inf

We can use the same tricks that the one used in the softmax computation observing the simple following fact: 
$$
\begin{aligned}
 tanh(x) &= \frac{e^{z} - e^{-z}}{e^{z} + e^{-z}} \\
 &= \left(\frac{e^{z} - e^{-z}}{e^{z} + e^{-z}}\right)\frac{e^{-a}}{e^{-a}} \\
 &= \frac{e^{z}e^{-a} - e^{-z}e^{-a}}{e^{z}e^{-a} + e^{-z}e^{-a}} \\
&= \frac{e^{z-a} - e^{-z-a}}{e^{z-a} + e^{-z-a}}
\end{aligned}
$$
Thus we want that $z-a$ or $-z-a$ be small, or in our case lower than $0$.  Thus taking $a$ as the absolute value of $z$ ($|z|$) will leads to have 
$z-a\leq 0$ and $-z-a\leq 0$.


For the backward notice that $tanh'(x) = 1-\sigma(x)^2$


In [8]:
class TanH(Operation):
    @staticmethod
    def TanHCompute(x):

        
        a = np.abs(x)
        tanh =  (np.exp(x+a) - np.exp(-x+a)) / (np.exp(x+a) + np.exp(-x+a))
        return tanh
        # return np.tanh(x)
        raise NotImplementedError
        
    @staticmethod
    def forward(x):
        # we copy the value of the input node
        np_x = x.value.copy()
        
        # we compute the tanh
        output_value = TanH.TanHCompute(np_x)

        # we create the output node needing only the node x
        output_node = ComputationGraphNode(np_x)
        output_node.set_input_nodes(x)

        return output_node
        raise NotImplementedError
        
    @staticmethod
    def backward(x, gradient):
        # we copy the value of the input node
        np_x = x.value.copy()

        # we compute the gradient
        grad = gradient * (1 - TanH.TanHCompute(np_x)**2)
        return (grad,)
        raise NotImplementedError
        

**Question 2:** Next, we implement the affine transform operation.
You can reuse the code from the third lab exercise, with one major difference: you have to compute the gradient with respect to x too!

In [9]:
class affine_transform(Operation):
    @staticmethod
    def forward(W, b, x):
        y = W.value @ x.value + b.value
        output_node = ComputationGraphNode(y)
        output_node.set_input_nodes(W, b, x)
        return output_node
        raise NotImplementedError
        
    @staticmethod
    def backward(W, b, x, gradient):
        
        
        # Calculate gradients for weights
        if x.value.ndim > 1:
            #
            grad_W = gradient.T @ x.value / gradient.shape[0]
            grad_x = gradient @ W.value
        else:
            grad_W = np.outer(gradient, x.value)
            grad_x = W.value.T @ gradient
    
        
    
        # Calculate gradients for biases
        grad_b = gradient.sum(axis=0) if gradient.ndim>1 else gradient
        
        return grad_W, grad_b, grad_x
        raise NotImplementedError

**Question 3:** Define the NLL operation

We recall that 
$$nll(x, y)= -log\left(\frac{e^{x_{y}}}{ \sum\limits_{i=1}^n e^{x_{ j}} }\right) = -x_{y} + log(\sum\limits_{i=1}^n e^{x_{ j} })$$

$$
    \begin{align*}
        \frac{\partial nll(x, y)}{\partial x_i} &= - \mathbb{1}_{y = i} + \frac{\partial log(\sum\limits_{i=1}^n e^{x_{ j} })}{\partial\sum\limits_{i=1}^n e^{x_{ j} }}\frac{\sum\limits_{i=1}^n e^{x_{ j} }}{\partial x_i} \\
        &= - \mathbb{1}_{y = i} + \frac{e^{x_i}}{\sum\limits_{i=1}^n e^{x_{ j} }} 
    \end{align*}
$$

In [10]:
class nll(Operation):
    @staticmethod
    def forward(x, y):
        # Convert x and y to appropriate shapes
        x_new = np.atleast_2d(x.value)
        y_new = np.atleast_1d(y.value).astype(int)

        # Apply softmax to input values
        sf = nll.softmax(x_new, axis=1)
    
        # Select the probabilities of the correct classes
        correct_class = sf[np.arange(len(sf)), y_new]
    
        # Calculate NLL loss
        nll_loss = -np.log(correct_class)
    
        # Adjust the output based on whether we have a single sample or a batch
        output_value = nll_loss[0] if len(nll_loss)==1 else nll_loss.mean()

        # Create and configure the output node
        output_node = ComputationGraphNode(output_value)
        output_node.set_input_nodes(x)
        output_node.set_func_parameters(y)
        return output_node
        raise NotImplementedError
        
    @staticmethod
    def backward(x, y, gradient):
        x_new = np.atleast_2d(x.value)
        y_new = np.atleast_1d(y.value).astype(int)
        # Compute softmax predictions
        sf = nll.softmax(x_new, axis=1)
    
        # Initialize gradient of x with softmax outputs
        grad_x = sf.copy()
    
        # Subtract 1 from the gradient of the correct classes
        grad_x[np.arange(len(sf)), y_new] -= 1
    
        # Adjust gradient for the number of samples
        factor = 1 if x_new.shape[0] == 1 else x_new.shape[0]
        grad_x *= (gradient / factor)
        # grad_x *= (gradient / max(x_new.shape[0], 1))
    
        # If the original input was a 1D array, flatten the output gradient
        if x.value.ndim == 1:
            grad_x = grad_x.flatten()
        return grad_x, None
        raise NotImplementedError
    
    @staticmethod
    def softmax(x, axis=None):
        exps = np.exp(x - np.max(x, axis=axis, keepdims=True))
        return exps / np.sum(exps, axis=axis, keepdims=True)

# Module

Neural networks or parts of neural networks will be stored in Modules.
They implement method to retrieve all parameters of the network and subnetwork.

In [11]:
class Module:
    def __init__(self):
        raise NotImplemented("")
        
    def parameters(self):
        ret = []
        for name in dir(self):
            o = self.__getattribute__(name)

            if type(o) is Parameter:
                ret.append(o)
            if isinstance(o, Module) or isinstance(o, ModuleList):
                ret.extend(o.parameters())
        return ret

# if you want to store a list of Parameters or Module,
# you must store them in a ModuleList instead of a python list,
# in order to collect the parameters correctly
class ModuleList(list):
    def parameters(self):
        ret = []
        for m in self:
            if type(m) is Parameter:
                ret.append(m)
            elif isinstance(m, Module) or isinstance(m, ModuleList):
                ret.extend(m.parameters())
        return ret

# Initialization and optimization

**Question 1:** Implement the different initialisation methods

In [21]:
def zero_init(b):
    b[:] = 0.

def glorot_init(W):
    '''inplace initialiase with glorot method'''
    W[:] = np.random.uniform(-np.sqrt(6. / (W.shape[0] + W.shape[1])), 
                             np.sqrt(6. / (W.shape[0] + W.shape[1])), 
                             W.shape)
    return W

    raise NotImplementedError
# Look at slides for the formula!
def kaiming_init(W):
    W[:] = np.random.uniform(-np.sqrt(6. / W.shape[0]), 
                             np.sqrt(6. / W.shape[0]), 
                             W.shape)
    return W
    raise NotImplementedError('Implement the initialization')

We will implement the Stochastic gradient descent through an object, in the init function this object will store the different parameters (in a list format). The step function will update the parameters (see slides), notice that the gradient is stored in the nodes (grad attribute). Finally it will be necessary after each update to reset all the gradient to zero (in the method zero_grad) because we do not want to accumumlate gradient of all previous step.

**Question 2:** Implement the SGD 

In [13]:
# simple gradient descent optimizer
class SGD:
    def __init__(self, params, lr=0.1):
        self.params = params
        self.lr = lr
        # raise NotImplementedError
        
    def step(self):
        for p in self.params:
            p.value -= self.lr * p.grad
        # raise NotImplementedError
        
    def zero_grad(self):
        for p in self.params:
            p.zero_grad()
        # raise NotImplementedError

# Networks and training loop

We first create a simple linear classifier, similar to the first lab exercise.

In [22]:
class LinearNetwork(Module):
    def __init__(self, dim_input, dim_output):
        # build the parameters
        self.W = Parameter(np.ndarray((dim_output, dim_input)), name="W")
        self.b = Parameter(np.ndarray((dim_output,)), name="b")

        self.init_parameters()
        # raise NotImplementedError
        
    def init_parameters(self):
        # init parameters of the network (i.e W and b)
        glorot_init(self.W.value)
        zero_init(self.b.value)
        # raise NotImplementedError
        
    def forward(self, x):
        # perform the forward pass
        return affine_transform()(self.W, self.b, x)
        raise NotImplementedError

In [23]:
np.random.seed(42)


In [24]:
# those lines should be executed correctly
lin1 = LinearNetwork(784, 10)
lin2 = LinearNetwork(10, 5)
x = ComputationGraphNode(train_data[0][0])
a = lin1.forward(x + x)
b = TanH()(a)
c = lin2.forward(b)

c.backward()
x.grad[0:10]

array([ 0.30770062, -0.12891701,  0.0547308 , -0.27474205,  0.29604521,
       -0.23439255,  0.05204224, -0.02639815, -0.12856254, -0.29513114])

We will train several neural networks.
Therefore, we encapsulate the training loop in a function.

**warning**: you have to call optimizer.zero_grad() before each backward pass to reinitialize the gradient of the parameters!

In [25]:
def training_loop(network, optimizer, train_data, dev_data, n_epochs=10):
    for epoch in range(n_epochs):
        # Initialize metrics for training and validation
        total_train_loss, correct_train_preds = 0, 0
        total_val_loss, correct_val_preds = 0, 0
        
        # Training Phase
        for x_train, y_train in zip(*train_data):
            optimizer.zero_grad()

            predictions_train = network.forward(ComputationGraphNode(x_train))
            loss = nll()(predictions_train, ComputationGraphNode(y_train))
            loss.backward()
            optimizer.step()

            total_train_loss += loss.value.mean()
            correct_train_preds += (np.argmax(predictions_train.value, axis=0) == y_train).sum()
        
        avg_train_loss = total_train_loss / train_data[0].shape[0]

        # Validation Phase
        for x_val, y_val in zip(*dev_data):
            predictions_val = network.forward(ComputationGraphNode(x_val))
            loss_val = nll()(predictions_val, ComputationGraphNode(y_val))

            total_val_loss += loss_val.value
            correct_val_preds += (np.argmax(predictions_val.value, axis=0) == y_val).sum()

        val_accuracy = correct_val_preds / dev_data[1].shape[0]
        avg_val_loss = total_val_loss / dev_data[0].shape[0]

        # Output the specified metrics: mean training loss and validation accuracy
        print(f"Epoch {epoch+1}/{n_epochs}: mean loss -> {avg_train_loss:.4f}, validation accuracy -> {val_accuracy:.6f}")


In [26]:
dim_input = 28*28
dim_output = 10

network = LinearNetwork(dim_input, dim_output)
optimizer = SGD(network.parameters(), 0.01)

training_loop(network, optimizer, train_data, dev_data)

Epoch 1/10: mean loss -> 0.3753, validation accuracy -> 0.920100
Epoch 2/10: mean loss -> 0.3098, validation accuracy -> 0.925900
Epoch 3/10: mean loss -> 0.2975, validation accuracy -> 0.925900
Epoch 4/10: mean loss -> 0.2906, validation accuracy -> 0.926000
Epoch 5/10: mean loss -> 0.2858, validation accuracy -> 0.926700
Epoch 6/10: mean loss -> 0.2822, validation accuracy -> 0.926700
Epoch 7/10: mean loss -> 0.2793, validation accuracy -> 0.927000
Epoch 8/10: mean loss -> 0.2769, validation accuracy -> 0.927300
Epoch 9/10: mean loss -> 0.2748, validation accuracy -> 0.927300
Epoch 10/10: mean loss -> 0.2730, validation accuracy -> 0.927600


After you finished the linear network, you can move to a deep network!

In [33]:
class DeepNetwork(Module):
    def __init__(self, dim_input, dim_output, hidden_dim, n_layers, tanh=False):
        
        self.layers = ModuleList()
        self.tanh = tanh
        self.n_layers = n_layers
        self.dim_input = dim_input
        self.dim_output = dim_output
        self.hidden_dim = hidden_dim


        # Initialize layers
        # Input to first hidden layer
        self.layers.append(LinearNetwork(dim_input, hidden_dim))
        
        # Hidden layers
        for i in range(1, n_layers):
            self.layers.append(LinearNetwork(hidden_dim, hidden_dim))

        # Last hidden to output layer
        self.layers.append(LinearNetwork(hidden_dim, dim_output))

        
        # Initialize parameters
        self.init_parameters()
        
    def init_parameters(self):
        # Initialize parameters for each layer
        for layer in self.layers:
            layer.init_parameters()
            
    def forward(self, x):
        '''Forward pass of the network'''
        for i in range(self.n_layers):
            x = self.layers[i].forward(x)
            if self.tanh:
                x = TanH()(x)
            else:
                x = ReLU()(x)
        x = self.layers[-1].forward(x)
        return x


In [34]:
dim_input = 28*28
dim_output = 10

network = DeepNetwork(dim_input, dim_output, 100, 2)
optimizer = SGD(network.parameters(), 0.01)

training_loop(network, optimizer, train_data, dev_data, n_epochs=5)

Epoch 1/5: mean loss -> 0.2433, validation accuracy -> 0.962100
Epoch 2/5: mean loss -> 0.1128, validation accuracy -> 0.968100
Epoch 3/5: mean loss -> 0.0798, validation accuracy -> 0.959900
Epoch 4/5: mean loss -> 0.0611, validation accuracy -> 0.967400
Epoch 5/5: mean loss -> 0.0476, validation accuracy -> 0.966200


## Better Optimizer
Implement the SGD with momentum, notice that you will need to store the cumulated gradient.


In [35]:
class SGDWithMomentum:
    def __init__(self, params, lr=0.1, momentum=0.5):
        self.params = params
        self.lr = lr
        self.momentum = momentum
        self.velocities = [np.zeros_like(p.value) for p in self.params] # Initialize velocities to zero

    def step(self):
        for i, (p, v) in enumerate(zip(self.params, self.velocities)):
            if p.grad is not None:  # Check if gradients are computed
                self.velocities[i] = self.momentum * v - self.lr * p.grad # Update velocity 
                p.value += self.velocities[i]  # Update parameter using gradients and velocities
            else:
                print(f"No gradient for parameter {p.name}")

        
    def zero_grad(self):
        for p in self.params:
            p.zero_grad()

In [36]:
dim_input = 28*28
dim_output = 10

network = DeepNetwork(dim_input, dim_output, 100, 3, tanh=False)
optimizer = SGDWithMomentum(network.parameters(), 0.01, momentum=0.4)

training_loop(network, optimizer, train_data, dev_data, n_epochs=10)

Epoch 1/10: mean loss -> 0.2923, validation accuracy -> 0.955400
Epoch 2/10: mean loss -> 0.1518, validation accuracy -> 0.962500
Epoch 3/10: mean loss -> 0.1168, validation accuracy -> 0.966400
Epoch 4/10: mean loss -> 0.0956, validation accuracy -> 0.958600
Epoch 5/10: mean loss -> 0.0850, validation accuracy -> 0.966300
Epoch 6/10: mean loss -> 0.0772, validation accuracy -> 0.964000
Epoch 7/10: mean loss -> 0.0681, validation accuracy -> 0.967200
Epoch 8/10: mean loss -> 0.0647, validation accuracy -> 0.964900
Epoch 9/10: mean loss -> 0.0716, validation accuracy -> 0.968600
Epoch 10/10: mean loss -> 0.0579, validation accuracy -> 0.970600


## Bonus: Batch SGD
Propose a methods to take into account batch of input

In [ ]:
def train_batch_sgd(model, optimizer, train_loader, epochs=10):
    for epoch in range(epochs):
        for batch_idx, (inputs, targets) in enumerate(train_loader):
            optimizer.zero_grad()  # 重置梯度
            outputs = model.forward(inputs)  # 前向传播
            loss = compute_loss(outputs, targets)  # 计算损失
            loss.backward()  # 反向传播，计算梯度
            optimizer.step()  # 更新模型参数

            if batch_idx % 100 == 0:
                print(f'Epoch {epoch}, Batch {batch_idx}, Loss: {loss.item()}')
